In [1]:
import torch
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
from deeplab_resnet import DeepLabv3_plus
from copy import deepcopy
import concurrent.futures
import time
import glob
import sys
%matplotlib inline

In [2]:
epoch = [0,1,3,4]
with open('val_v_5.json','r') as f:
    val_file = json.load(f)
val_file = json.loads(val_file)

In [3]:
def compute_dismap(dismap, bbox):
    x_min, y_min, x_max, y_max = bbox[:]

    # draw bounding box
    cv2.line(dismap, (x_min, y_min), (x_max, y_min), color=1, thickness=1)
    cv2.line(dismap, (x_min, y_min), (x_min, y_max), color=1, thickness=1)
    cv2.line(dismap, (x_max, y_max), (x_max, y_min), color=1, thickness=1)
    cv2.line(dismap, (x_max, y_max), (x_min, y_max), color=1, thickness=1)

    tmp = (dismap > 0).astype(np.uint8) # mark boundary
    tmp_ = deepcopy(tmp)

    fill_mask = np.ones((tmp.shape[0] + 2, tmp.shape[1] + 2)).astype(np.uint8)
    fill_mask[1:-1, 1:-1] = tmp_
    cv2.floodFill(tmp_, fill_mask, (int((x_min + x_max) / 2), int((y_min + y_max) / 2)), 5) # fill pixel inside bounding box

    tmp_ = tmp_.astype(np.int8)
    tmp_[tmp_ == 5] = -1 # pixel inside bounding box
    tmp_[tmp_ == 0] = 1 # pixel on and outside bounding box

    tmp = (tmp == 0).astype(np.uint8)

    dismap = cv2.distanceTransform(tmp, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)  # compute distance inside and outside bounding box
    dismap = tmp_ * dismap + 128

    dismap[dismap > 255] = 255
    dismap[dismap < 0] = 0

    dismap = dismap.astype(np.uint8)

    return dismap

In [4]:
def distance_map(mask,bbox):
    bounds = (0, 0, mask.shape[1] - 1, mask.shape[0] - 1)


    x_min = max(bbox[0], bounds[0])
    y_min = max(bbox[1], bounds[1])
    x_max = min(bbox[2], bounds[2])
    y_max = min(bbox[3], bounds[3])

    bbox = [x_min, y_min, x_max, y_max]

    dismap = np.zeros((mask.shape[0], mask.shape[1]))
    dismap = compute_dismap(dismap, bbox)
    return dismap

In [8]:
thres = 0.75
def get_val_result(sample):
    select_model_path, key, category = sample
    net = DeepLabv3_plus(n_classes=1, nInputChannels=4)
    model_name = select_model_path
    checkpoint = torch.load(model_name, map_location=lambda storage, loc: storage)
    pretrained_dict = {k: v for k, v in checkpoint.items() if k in net.state_dict()}
    model_dict = net.state_dict()
    model_dict.update(pretrained_dict)
    net.load_state_dict(model_dict)
    net.eval()
    base_dir = '/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/'
    if not os.path.exists(base_dir):
        os.mkdir(base_dir)
    result_dir = os.path.join(base_dir, key)
    if not os.path.exists(result_dir):
        os.mkdir(result_dir)
    temp = val_file[key]
    for frame in temp:
        img_path = temp[frame]['img_path']
        img = cv2.imread(img_path)
        result_shape = img.shape
        if category in temp[frame].keys():
            mask_dir = os.path.join(result_dir, category)
            if not os.path.exists(mask_dir):
                os.mkdir(mask_dir)
            bbox = temp[frame][category]['bbox']
            dis_map = distance_map(img, bbox)
            dis_map = cv2.resize(dis_map, (450, 450), interpolation=cv2.INTER_NEAREST)
            dis_map = dis_map[:,:, np.newaxis]
            res = cv2.resize(img, (450,450), interpolation=cv2.INTER_CUBIC)
            res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
            res = np.concatenate((res, dis_map), axis=2)
            res = res.transpose((2,0,1))
            res = res[np.newaxis,:,:,:]
            res = torch.from_numpy(res).float()
            outputs = net.forward(res)
            outputs = outputs.to(torch.device('cpu'))
            prediction = np.transpose(outputs.data.numpy()[0, ...], (1, 2, 0))
            prediction = 1 / (1 + np.exp(-prediction))
            prediction = np.squeeze(prediction)
            prediction[prediction>thres] = 255
            prediction[prediction<=thres] = 0
            prediction = prediction.astype(np.uint8)
            prediction = cv2.resize(prediction, (result_shape[1], result_shape[0]),interpolation=cv2.INTER_NEAREST)
            result_path  = os.path.join(mask_dir, img_path.split('/')[-1].replace('.jpg', '.png'))
            print(result_path)
            cv2.imwrite(result_path, prediction)

In [9]:
num = 0
base_dir = '/home/ningxinL/model_zoo/DeepGrabCut/finetune_models/'
sample_list = []
for root,directions, files in os.walk(base_dir):
    for direction in directions:
        try:
            category, instance_num = direction.split('_')
        except:
            continue
        num +=1
        model_list = glob.glob(os.path.join(base_dir, direction)+'/*.pth')
        max_epoch_num = 0
        for model_path in model_list:
            epoch_num = int((model_path.split('-')[-1]).split('_')[0])
            #if epoch_num > max_epoch_num:
            if epoch_num == 1:
                #max_epoch_num = epoch_num
                select_model_path = model_path
                break
        sample = [select_model_path, category, instance_num]
        sample_list.append(sample)

In [10]:
with concurrent.futures.ProcessPoolExecutor() as executor:
        executor.map(get_val_result, sample_list)
sys.exit()

Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Number of classes: 1
Number of classes: 1
Number of classes: 1
Output stride: 16
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Output stride: 16
Constructing DeepLabv3+ model...
Output stride: 16
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Number of classes: 1
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Number of Input Channels: 4
Number of classes: 1
Number of classes: 1
Number of classes: 1
Number of classes: 1
Number of Input Channels: 4
Number of classes: 1
Number of Input Channels: 4
Number of classes: 1
Number of classes: 1
Output stride: 16
Number of classes: 1
Number of classes: 1
Number of classes: 1
Number of classes: 1
O

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in exp


/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/india/2/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/shooting/3/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/gold-fish/2/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/motocross-jump/2/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/paragliding-launch/1/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/gold-fish/5/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/judo/2/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/drift-chicane/1/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/gold-fish/3/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/shooting/1/00007.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_fi

/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/horsejump-high/2/00010.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/drift-chicane/1/00036.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/paragliding-launch/3/00053.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/india/2/00036.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/motocross-jump/2/00015.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/shooting/1/00015.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/bmx-trees/1/00069.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/shooting/3/00015.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/gold-fish/2/00069.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/india/1/00038.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_

/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/bmx-trees/1/00048.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/gold-fish/5/00020.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/gold-fish/3/00020.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/gold-fish/2/00020.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/paragliding-launch/3/00048.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/shooting/3/00010.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/paragliding-launch/1/00048.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/judo/1/00016.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/lab-coat/1/00004.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/drift-chicane/1/00042.png
/home/ningxinL/DeepGrabCut_Davis_val/result/d

...

/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/camel/1/00083.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/3/00043.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/soapbox/3/00084.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/india/3/00065.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/mbike-trick/2/00043.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/bmx-trees/2/00033.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/2/00010.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/kite-surf/2/00037.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/dogs-jump/3/00020.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/1/00031.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/lab-coat/5/

/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/lab-coat/5/00008.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/3/00002.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/soapbox/3/00089.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/india/3/00006.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/mbike-trick/2/00002.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/2/00004.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/dogs-jump/3/00044.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/1/00078.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/lab-coat/5/00000.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/dog/1/00016.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/3/00030.pn

/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/mbike-trick/2/00005.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/2/00023.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/india/3/00032.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/1/00060.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/3/00003.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/dog/1/00057.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/dogs-jump/3/00024.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/mbike-trick/2/00003.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/india/3/00060.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/2/00022.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/1/00008.png
/

/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/3/00033.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/mbike-trick/2/00033.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/2/00011.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/dogs-jump/3/00016.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/3/00050.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/mbike-trick/2/00050.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/2/00067.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/dogs-jump/3/00065.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/3/00051.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/mbike-trick/2/00051.png
/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_epoch_1_v_5/pigs/2/

NameError: name 'sys' is not defined